In [ ]:
# import settings and functions
%run ./../../dataset_creation_imports.ipynb

## What mesh?

Copy your choice to the next cell

for SquareTop:
```
analytical_solution_tag = "-ana_square_top"
generate_config = generateConfig_squareTop
generate_mesh = generateMesh_squareTop
```

for SquareSinCos:
```
analytical_solution_tag = "-ana_square_sincos"
generate_config = generateConfig_squareSinCos
generate_mesh = generateMesh_squareSinCos
```

In [ ]:
# Change according to instruction above
analytical_solution_tag = "-ana_square_sincos"
generate_config = generateConfig_squareSinCos
generate_mesh = generateMesh_squareSinCos

# analytical_solution_tag = "-ana_L_shape"
# generate_config = generateConfig_Lshape
# generate_mesh = generateMesh_Lshape

# analytical_solution_tag = "-ana_mexi_hat"
# generate_config = generateConfig_squareMexiHat
# generate_mesh = generateMesh_squareMexiHat

# analytical_solution_tag = "-ana_square_top"
# generate_config = generateConfig_squareTop
# generate_mesh = generateMesh_squareTop

In [ ]:
exe = data_driven_diffusion_snes
prefix = "scale_DD_"

# exe = hdiv_data_driven_diffusion_snes
# prefix = "DD_hdiv_"

how_many_values = [11, 101, 1001, 10001]
# how_many_values = [11, 101, 1001]

run_simulation = True
run_line_simulation = True

run_simulation = False
run_line_simulation = False


In [ ]:
how_many = 41
#
params.conductivity = 1.0 # linear conductivity
params.element_size = 0.05 # element size in the regular mesh
params.order = 2 # approximation order for displacements
# Pre-processing parameters
params.mesh_file = "square_top"
params.length_x = 1
params.length_y = 1

params.nproc = 1 # number of processors

In [ ]:
# start display for showing results
display = Display(backend="xvfb", visible=False, size=(1024, 768))
display.start()

In [ ]:
# Testing mesh generation

params.show_mesh = True
generate_config(params)
generate_mesh(params)

In [ ]:
dataset_name = "grid_data.csv"
make_grid_linear_dataset(8, how_many, dataset_name, 0.1, plot_dataset=True)

In [ ]:

k_in_list = np.logspace(np.log10(0.1), np.log10(10), 51)
print(k_in_list)

In [ ]:
use_line = "-use_line"
# use_line = ""

if use_line:
    prefix += "line_"
    print(prefix)
# prefix += 

In [ ]:
# Testing running analysis
!rm out_*h5m

how_many = 1001

number_of_k = 51
k_in_list = np.logspace(np.log10(0.05), np.log10(50), number_of_k)
sumana_name = "sumanalys_" + prefix + str(number_of_k) 
print(sumana_name)

# make_grid_linear_dataset(8, how_many, dataset_name, 0.1, plot_dataset=False)

params.part_file = params.mesh_file + "_" + str(params.nproc) + "p.h5m"
!{mofem_part} -my_file {params.mesh_file + ".h5m"} -my_nparts {params.nproc} -output_file {params.part_file} -dim 2 -adj_dim 1
if run_simulation:
    !rm sumanalys.csv
    for k_in in k_in_list:
        make_grid_linear_dataset(8, how_many, dataset_name, k_in, plot_dataset=False)
        print("k_in = ", k_in)
        !{exe} -file_name {params.part_file} -my_order {params.order} {analytical_solution_tag} -csv_tree_file {dataset_name} {use_line} -scaling 0 -my_dummy_k {k_in} -snes_max_it 5000
        print("analysis done")
    !cp sumanalys.csv {sumana_name}.csv

    # !{exe} -file_name {params.part_file} -my_order {params.order} {analytical_solution_tag} -csv_tree_file {dataset_name} -write_long_error_file {use_line} -print_integ -scaling 0 -my_dummy_k 0.1  -snes_max_it 5000
# -use_line -snes_max_it 500

out_to_vtk = !ls -c1 out_result_*h5m

!convert.py {out_to_vtk[0]}
!rm out_*h5m

In [ ]:

naming = ["order", "gaussnum", "iterations","volume", "datanum","rmsPoiErr", "errorEstimator",
          "L2norm", "H1seminorm","fluxErr", "orderRefinementCounter", "errorIndicatorGrad", "errorIndicatorDiv", "jumpL2", "jumpHdiv", "eleNum"]

error_name_list = ["L2norm", "H1seminorm", "fluxErr"]
error_label_list = [(r'Global error $L^2$-norm'),
               (r'Global error $H^1$-seminorm'), (r'Global Flux error')]

# error_name_list = ["iterations", "rmsPoiErr"]
# error_label_list = [(r'Global error $L^2$-norm'),
#                (r'Global error $H^1$-seminorm'), (r'Global Flux error')]

# iter_ana = Analysis("sumanalys", naming, [], [], [], prefix, [],  marker='x', linestyle='--', plot_gradients=True, label="Mixed")
iter_ana = Analysis(sumana_name, naming, [], [], [], prefix, [],  marker='x', linestyle='--', plot_gradients=True, label="Mixed")

In [ ]:
# print(iter_ana.data["fluxErr"])

# # divide each iter_ana.data["fluxErr"] by k_in_list
# for i in range(len(k_in_list)):
#     iter_ana.data["fluxErr"][i] /= k_in_list[i]

# print(iter_ana.data["fluxErr"])

In [ ]:
plt.figure()

plt.plot(k_in_list, iter_ana.data["L2norm"], label="No scaling", marker='x', linestyle='--')

plt.xlabel(label_conductivity)
plt.ylabel(r'Global error $L^2$-norm')
plt.xscale('log')
plt.yscale('log')
ax = plt.gca()  # Define ax variable
ax.grid(True, ls=':')  # Call ax.grid() method
plt.legend()
plt.tight_layout()
plt.savefig(prefix+'scaling_l2_err.pdf')
plt.show()

plt.figure()
plt.plot(k_in_list, iter_ana.data["H1seminorm"], label="No scaling", marker='x', linestyle='--')
plt.xlabel(label_conductivity)
plt.ylabel(r'Global error $H^1$-seminorm')
plt.xscale('log')
plt.yscale('log')
ax = plt.gca()  # Define ax variable
ax.grid(True, ls=':')  # Call ax.grid() method
plt.legend()
plt.tight_layout()
plt.savefig(prefix+'scaling_h1_err.pdf')
plt.show()

plt.figure()
plt.plot(k_in_list, iter_ana.data["fluxErr"], label="No scaling", marker='x', linestyle='--')
plt.xlabel(label_conductivity)
plt.ylabel(r'Global Flux error')
plt.xscale('log')
plt.yscale('log')
ax = plt.gca()  # Define ax variable
ax.grid(True, ls=':')  # Call ax.grid() method
plt.legend()
plt.tight_layout()
plt.savefig(prefix+'scaling_flux_err.pdf')


In [ ]:
plt.figure(figsize=(12, 4))

plt.plot(k_in_list, iter_ana.data["iterations"], label="No scaling", marker='x', linestyle='--')

plt.xlabel(label_conductivity)
plt.ylabel("Number of iterations")
plt.xscale('log')
plt.yscale('log')
ax = plt.gca()  # Define ax variable
ax.grid(True, ls=':')  # Call ax.grid() method
plt.legend()
plt.tight_layout()
plt.savefig(prefix+'scaling_iterations.pdf')
plt.show()

In [ ]:

params.field_part = -1
params.show_file = "out_iteration_"
params.show_file = "out_result_"
params.show_field = "T"
# params.warp_factor = 0.4  # warp factor
params.show_edges = True
# params.p_save = "run_test_p.pdf"
show_results(params)

In [ ]:
params.show_field = "Q"
params.field_part = 1
show_results(params)